In [1]:
from pprint import pprint
import os

# Read results files

In [6]:
# # assumes we're in {app}-{version}/ with 
# # {system_name}-{compute_instance_type}/{compiler_family}-{mpi_family}/runs/{run_label/}run.json
# #
# # could probably do with abstracting this out - everything will need it!

# # TODO: simplify this - e.g. just pull out path name then split that into lables

# def fs_tree(root='.'):
#     """ Return a nested dict of the filetree below `root`.
    
#         Directory names are represented as keys.
#         File names are available under the key "./".
#     """
#     results = {}
#     for (dirpath, dirnames, filenames) in os.walk(root):
#         # in-place filter of dirnames to avoid hidden directories:
#         for idx, d in enumerate(dirnames):
#             if d.startswith('.'):
#                 del dirnames[idx]
#         parts = dirpath.split(os.sep)
#         if parts[0] == '.':
#             parts = parts[1:]
#         curr = results
#         for p in parts:
#             curr = curr.setdefault(p, {'./':filenames})
#     return results

# TODO: function to separate path names into lables:
#          'system_name'
#                 'compute_instance_type'
#                 'compiler_family'
#                 'mpi_family'
#                 'run_label'
#                 'filename'
#                    fpath = os.path.join(* parts + ['runs', rundir, runfile])
#                     lables = {'system_name':parts[0].split('-')[0],
#                               'compute_instance_type':parts[0].split('-')[1],
#                               'compiler_family':parts[1].split('-')[0],
#                               'mpi_family':parts[1].split('-')[1],
#                               'run_label':rundir,
#                               'filename':runfile,
#                              }

def find_run_outputs(root='.', ext='.out'):
    """ Find all files with matching extensions.
    
        Returns a sequence of paths.
    """
    results = []
    for (dirpath, dirnames, filenames) in os.walk(root):
        # in-place filter dirnames to avoid hidden directories:
        for idx, d in enumerate(dirnames):
            if d.startswith('.'):
                del dirnames[idx]
        for f in filenames:
            if os.path.splitext(f)[-1] == ext:
                pathparts = [root] + dirpath.split(os.path.sep) + [f]
                # if it's a run output then path is ".../runs/{run_label}/{filename}
                if pathparts[-3] == 'runs':
                    fullpath = os.path.normpath(os.path.join(*pathparts))
                    results.append(fullpath)
    return(results)
                
run_outs = find_run_outputs()
print run_outs

['sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-13.out', 'sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-14.out', 'sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-15.out', 'sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-16.out']


In [14]:
def read_imb_out(path):
    results = {'file':path}
    COL_TYPES = {'Uniband':(int, int, float, int),
                 'Biband':(int, int, float, int),
                }
    with open(path) as f:
        for line in f:
            if line.startswith('# Benchmarking '):
                benchmark = line.split()[-1]
                processes = next(f).split()[-1] # "# #processes = 2"
                result = {'name':benchmark, 'data':{}, 'params':{'processes':processes}}
                results[benchmark] = result
                next(f) # skip header
                while True:
                    cols = next(f).split()
                    if cols == []:
                        break
                    if cols[0].startswith('#'): # header row
                        header = cols
                        for label in header:
                            result['data'][label] = []
                    else:
                        for label, opr, value in zip(header, COL_TYPES[benchmark], cols):
                            result['data'][label].append(opr(value))
    return results

results = []
for path in run_outs:
    print 'read', path
    results.append(read_imb_out(path))


read sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-13.out
read sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-14.out
read sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-15.out
read sausage-hotdog/gnu8-openmpi3/runs/N2npn1/slurm-16.out


In [12]:
pprint(results)

[{},
 {},
 {},
 {'Biband': {'data': {'#bytes': [0,
                                 1,
                                 2,
                                 4,
                                 8,
                                 16,
                                 32,
                                 64,
                                 128,
                                 256,
                                 512,
                                 1024,
                                 2048,
                                 4096,
                                 8192,
                                 16384,
                                 32768,
                                 65536,
                                 131072,
                                 262144,
                                 524288,
                                 1048576,
                                 2097152,
                                 4194304],
                      '#repetitions': [1000,
         

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
slurm16 = results[-1]
biband = slurm16['Biband']['data']
uniband = slurm16['Uniband']['data']
plt.plot(biband['#bytes'], biband['Mbytes/sec'], label='biband')
plt.plot(uniband['#bytes'], uniband['Mbytes/sec'], label='uniband')
plt.grid()
plt.title(slurm16['file'])
plt.legend()
plt.xscale('log')
plt.xlabel('#bytes')
plt.ylabel('Mbytes/sec')


Text(0,0.5,'Mbytes/sec')